In [1]:
# Standard Library Imports
import json
from pathlib import Path


# Third-Party Libraries
import numpy as np
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, models, transforms

# Avalanche: Continual Learning Framework
## Benchmarks
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.datasets.torchvision_wrapper import CIFAR10
from avalanche.benchmarks.scenarios import CLExperience
from avalanche.benchmarks.utils.flat_data import ConstantSequence

## Models
from avalanche.models import (
    MultiHeadClassifier,
    MultiTaskModule,
    MTSimpleMLP,
    MTSimpleCNN,
    PNN,
)

## Training Strategies
from avalanche.training.supervised import Naive, EWC, LwF

## Plugins and Logging
from avalanche.logging import InteractiveLogger, TextLogger
from avalanche.training.plugins import EvaluationPlugin, LRSchedulerPlugin

## Evaluation Metrics
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics,
    loss_metrics,
    timing_metrics,
    cpu_usage_metrics,
    confusion_matrix_metrics,
    disk_usage_metrics,
)

import random

In [2]:
SAVE = False
import os

if SAVE:
    os.chdir('/home/uregina/DL_Project')
    print(os.getcwd())

# For saving the datasets/models/results/log files

if SAVE:
    DATASET_NAME = "SplitCIFAR10"
    ROOT = Path("/home/uregina/DL_Project")
    DATA_ROOT = ROOT / DATASET_NAME
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed = 0

DATASET_NAME = "SplitCIFAR10"
NUM_CLASSES = {
    "SplitCIFAR10": 10
}

# Define hyperparameters/scheduler/augmentation
HPARAM = {
    "batch_size": 128,        #CHANGE
    "num_epoch": 3,           #CHANGE
    "start_lr": 0.01,
    "alpha": 0.7,
    "temperature": 2,
}

In [4]:
from avalanche.models.dynamic_modules import DynamicModule

class ResNetIncrementalCNNClassifier(DynamicModule):
    """
    Output layer that incrementally adds units whenever new classes are
    encountered.

    Typically used in class-incremental benchmarks where the number of
    classes grows over time.
    """

    def __init__(self, in_features, initial_out_features=2, masking=True, mask_value=-10000000):
        """
        :param in_features: number of input features.
        :param initial_out_features: initial number of classes (can be
            dynamically expanded).
        :param masking: whether unused units should be masked (default=True).
        :param mask_value: the value used for masked units (default=-1000).
        """
        super().__init__()
        self.masking = masking
        self.mask_value = mask_value

        self.features = nn.Sequential(
            # First convolution: 192 input channels to 256 output channels
            nn.Conv2d(192, 256, kernel_size=3, stride=1, padding=1),  # Maintain spatial resolution
            nn.BatchNorm2d(256),  # Add BatchNorm for stability
            nn.ReLU(),

            # Second convolution: 256 input channels to 128 output channels
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),  # Still maintain spatial resolution
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # Additional layer: 128 input channels to 128 output channels
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),  # Enhance feature extraction
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # Third convolution: 128 input channels to 64 output channels
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),  # Spatial resolution retained
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # Additional layer: 64 input channels to 66 output channels
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # Slightly increase channels
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # Global Average Pooling for spatial size reduction
            nn.AdaptiveAvgPool2d((1, 1)),

            # Dropout for regularization
            nn.Dropout(p=0.25)
        )
        
        # Classifier will take the 64 channels (final output from features)
        self.classifier = nn.Linear(64, initial_out_features)
 

        au_init = torch.zeros(initial_out_features, dtype=torch.bool)
        self.register_buffer('active_units', au_init)

    @torch.no_grad()
    def adaptation(self, experience: CLExperience):
        """If `dataset` contains unseen classes the classifier is expanded.

        :param experience: data from the current experience.
        :return:
        """
        in_features = self.classifier.in_features
        old_nclasses = self.classifier.out_features
        curr_classes = experience.classes_in_this_experience
        #print("curr_classes", old_nclasses, curr_classes)
        new_nclasses = max(
            self.classifier.out_features, max(curr_classes) + 1
        )

        # update active_units mask
        if self.masking:
            if old_nclasses != new_nclasses:  # expand active_units mask
                old_act_units = self.active_units
                self.active_units = torch.zeros(new_nclasses, dtype=torch.bool)
                self.active_units[:old_act_units.shape[0]] = old_act_units
            # update with new active classes
            if self.training:
                self.active_units[curr_classes] = 1

        # update classifier weights
        if old_nclasses == new_nclasses:
            return
        old_w, old_b = self.classifier.weight, self.classifier.bias
        
        self.classifier = torch.nn.Linear(in_features, new_nclasses)
        self.classifier.weight[:old_nclasses] = old_w
        self.classifier.bias[:old_nclasses] = old_b

    def forward(self, x, **kwargs):
        """Compute the output given the input `x`. This module does not use
        the task label.

        :param x:
        :return:
        """
        x  = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the output for classifier
        out = self.classifier(x)
        if self.masking:
            out[..., torch.logical_not(self.active_units)] = self.mask_value
        return out



class ResNetMultiHeadCNNClassifier(MultiTaskModule):
    """Multi-head classifier with separate heads for each task.

    Typically used in task-incremental benchmarks where task labels are
    available and provided to the model.

    .. note::
        Each output head may have a different shape, and the number of
        classes can be determined automatically.

        However, since pytorch doest not support jagged tensors, when you
        compute a minibatch's output you must ensure that each sample
        has the same output size, otherwise the model will fail to
        concatenate the samples together.

        These can be easily ensured in two possible ways:

        - each minibatch contains a single task, which is the case in most
            common benchmarks in Avalanche. Some exceptions to this setting
            are multi-task replay or cumulative strategies.
        - each head has the same size, which can be enforced by setting a
            large enough `initial_out_features`.
    """

    def __init__(self, in_features, initial_out_features=2,
                 masking=True, mask_value=-10000000):
        """Init.

        :param in_features: number of input features.
        :param initial_out_features: initial number of classes (can be
            dynamically expanded).
        :param masking: whether unused units should be masked (default=True).
        :param mask_value: the value used for masked units (default=-1000).
        """
        super().__init__()
        self.masking = masking
        self.mask_value = mask_value
        self.in_features = in_features
        self.starting_out_features = initial_out_features
        self.classifiers = torch.nn.ModuleDict()

        # needs to create the first head because pytorch optimizers
        # fail when model.parameters() is empty.
        # masking in IncrementalClassifier is unaware of task labels
        # so we do masking here instead.
        first_head = ResNetIncrementalCNNClassifier(
            self.in_features, self.starting_out_features, masking=False
        )
        self.classifiers["0"] = first_head
        self.max_class_label = max(self.max_class_label, initial_out_features)

        au_init = torch.zeros(initial_out_features, dtype=torch.bool)
        self.register_buffer('active_units_T0', au_init)


    def adaptation(self, experience: CLExperience):
        """If `dataset` contains new tasks, a new head is initialized.

        :param experience: data from the current experience.
        :return:
        """
        super().adaptation(experience)
        curr_classes = experience.classes_in_this_experience
        task_labels = experience.task_labels
        if isinstance(task_labels, ConstantSequence):
            # task label is unique. Don't check duplicates.
            task_labels = [task_labels[0]]

        for tid in set(task_labels):
            # head adaptation
            tid = str(tid)  # need str keys
            if tid not in self.classifiers:  # create new head
                new_head = ResNetIncrementalCNNClassifier(
                    self.in_features, self.starting_out_features
                )
                self.classifiers[tid] = new_head

                au_init = torch.zeros(self.starting_out_features,
                                      dtype=torch.bool)
                self.register_buffer(f'active_units_T{tid}', au_init)

            self.classifiers[tid].adaptation(experience)

            # update active_units mask for the current task
            if self.masking:
                # TODO: code below assumes a single task for each experience
                # it should be easy to generalize but it may be slower.
                if len(task_labels) > 1:
                    raise NotImplementedError(
                        "Multi-Head unit masking is not supported when "
                        "experiences have multiple task labels. Set "
                        "masking=False in your "
                        "MultiHeadClassifier to disable masking.")

                au_name = f'active_units_T{tid}'
                curr_head = self.classifiers[str(tid)]
                old_nunits = self._buffers[au_name].shape[0]

                new_nclasses = max(
                    curr_head.classifier.out_features, max(curr_classes) + 1
                )
                if old_nunits != new_nclasses:  # expand active_units mask
                    old_act_units = self._buffers[au_name]
                    self._buffers[au_name] = torch.zeros(new_nclasses,
                                                         dtype=torch.bool)
                    self._buffers[au_name][:old_act_units.shape[0]] = \
                        old_act_units
                # update with new active classes
                if self.training:
                    self._buffers[au_name][curr_classes] = 1

    def forward_single_task(self, x, task_label):
        """compute the output given the input `x`. This module uses the task
        label to activate the correct head.

        :param x:
        :param task_label:
        :return:
        """
        out = self.classifiers[str(task_label)](x)
        if self.masking:
            au_name = f'active_units_T{task_label}'
            curr_au = self._buffers[au_name]
            nunits, oldsize = out.shape[-1], curr_au.shape[0]
            if oldsize < nunits:  # we have to update the mask
                old_mask = self._buffers[au_name]
                self._buffers[au_name] = torch.zeros(nunits, dtype=torch.bool)
                self._buffers[au_name][:oldsize] = old_mask
                curr_au = self._buffers[au_name]
            out[..., torch.logical_not(curr_au)] = self.mask_value
        return out

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn

class EfficientNetPCNN(MultiTaskModule):
    def __init__(self):
        super(EfficientNetPCNN, self).__init__()
        # Use the updated `weights` parameter instead of `pretrained`
        self.model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        
        # Freeze all parameters in the model
        for param in self.model.parameters():
            param.requires_grad = False
        
        # Extract layers up to block 7
        self.block7 = nn.Sequential(
            *list(self.model.features[:7])  # Extract up to and including block 7
        )
        
        # Custom classifier
        self.classifier = ResNetMultiHeadCNNClassifier(192)

        # Freeze EfficientNet parameters
        for param in self.model.parameters():
            param.requires_grad = False
    
    def forward_single_task(self, x, task_id):
        x = self.block7(x)
        out = self.classifier(x, task_id)
        return out

In [5]:
import torchvision.transforms as transforms

# print to stdout
interactive_logger = InteractiveLogger()

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
train_transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)
test_transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

benchmark = SplitCIFAR10(
    n_experiences = 5,          #CHANGE
    return_task_id = True,
    seed=seed,
    train_transform=train_transform,
    eval_transform=test_transform
)

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(
        num_classes=NUM_CLASSES[DATASET_NAME], save_image=False, stream=True
    ),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=interactive_logger,
)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
MODEL_NAME = 'EfficientNetPCNN'
RUN = '0'                    #Multiple runs 0,1,2
model = EfficientNetPCNN()

optimizer = Adam(model.parameters(), HPARAM["start_lr"])

cl_strategy = LwF(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=HPARAM["batch_size"],
    train_epochs=HPARAM["num_epoch"],
    eval_mb_size=HPARAM["batch_size"],
    alpha=HPARAM["alpha"],              # LwF parameter
    temperature=HPARAM["temperature"],  # LwF parameter
    evaluator=eval_plugin,
    device=device,
)

if SAVE:
    DATA_ROOT = ROOT / DATASET_NAME / MODEL_NAME / RUN
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

In [7]:
print("Starting experiment...")
results_dict = {}  # Use a dictionary instead of a list
for index, experience in enumerate(benchmark.train_stream):
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    res = cl_strategy.train(experience)
    print("Training completed")
    print("Computing accuracy on the whole test")
    results_dict[index] = cl_strategy.eval(benchmark.test_stream)  # Use the index as the key

print("Experiment completed")

Starting experiment...
Start of experience:  0
Current Classes:  [1, 4]
-- >> Start of training phase << --
 25%|██▌       | 20/79 [00:07<00:20,  2.93it/s]

KeyboardInterrupt: 

In [ ]:
if SAVE:
    file_name = f"{MODEL_NAME}_{DATASET_NAME}_{RUN}_results.txt"
    file_path = ROOT / DATASET_NAME / MODEL_NAME / RUN / file_name
    with open(file_path, "w") as file:
        file.write(f"Model: {MODEL_NAME}\n")
        file.write(f"Dataset: {DATASET_NAME}\n")
        file.write(f"Run: {RUN}\n") 
        file.write("\nResults Dictionary:\n")
        file.write("--------------------------------------------------\n")
        for key, value in results_dict.items():
            file.write(f"Experience {key}:\n")
            for metric, metric_value in value.items():
                # Convert tensors to lists for saving
                if isinstance(metric_value, torch.Tensor):
                    metric_value = metric_value.tolist()
                file.write(f"  {metric}: {metric_value}\n")
            file.write("--------------------------------------------------\n")